In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
lines=sc.parallelize(['it is fun to have fun','but you have to know how'])

###PySpark Word Count Program

In [3]:
wordcounts=lines.map(lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
           .flatMap(lambda x: x.split()) \
           .map(lambda x: (x,1)) \
           .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(ascending=False)

In [4]:
wordcounts.take(20)

[(2, 'have'),
 (2, 'to'),
 (2, 'fun'),
 (1, 'but'),
 (1, 'you'),
 (1, 'how'),
 (1, 'is'),
 (1, 'know'),
 (1, 'it')]

In [1]:
telco_churn=spark.read.csv("D:/TelcoChurn.csv", header=True,inferSchema=True)

## Instead of using Spark Context we can directly use like above in the 2.2.0 version of PySpark
## This will read the file and the variable correctly in their format like in R language

In [2]:
telco_churn.printSchema()

root
 |-- Account Length: integer (nullable = true)
 |-- International Plan: integer (nullable = true)
 |-- Voice Mail Plan: integer (nullable = true)
 |-- Num of Voice mail Messages: integer (nullable = true)
 |-- Total Day Minutes: double (nullable = true)
 |-- Total Day Calls: integer (nullable = true)
 |-- Total day Charge: double (nullable = true)
 |-- Total Eve Minutes: double (nullable = true)
 |-- Total Eve Calls: integer (nullable = true)
 |-- Total Eve Charge: double (nullable = true)
 |-- Total Night Minutes: double (nullable = true)
 |-- Total Night Calls : integer (nullable = true)
 |-- Total Night Charge: double (nullable = true)
 |-- Total International Minutes: double (nullable = true)
 |-- Total Intl  Calls: integer (nullable = true)
 |-- Total Intl Charge: double (nullable = true)
 |-- Number Customer Service calls : integer (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
insur_data=spark.read.csv("D:/analytics/Insurance_Data.csv",header=True,inferSchema=True)

In [5]:
insur_data.printSchema()

root
 |-- Policy Number: integer (nullable = true)
 |-- Number of Vehicles: integer (nullable = true)
 |-- Average Age: double (nullable = true)
 |-- Gender Dummy: integer (nullable = true)
 |-- Married Dummy: integer (nullable = true)
 |-- Avg Veh Age: double (nullable = true)
 |-- Fuel Type Dummy: integer (nullable = true)
 |-- Losses: double (nullable = true)



In [6]:
insur_data.take(10)

[Row(Policy Number=150023, Number of Vehicles=1, Average Age=20.6, Gender Dummy=1, Married Dummy=0, Avg Veh Age=12.97, Fuel Type Dummy=0, Losses=203.3195946),
 Row(Policy Number=152720, Number of Vehicles=1, Average Age=42.4, Gender Dummy=0, Married Dummy=1, Avg Veh Age=8.02, Fuel Type Dummy=0, Losses=402.1973024),
 Row(Policy Number=174207, Number of Vehicles=1, Average Age=42.4, Gender Dummy=1, Married Dummy=0, Avg Veh Age=2.5, Fuel Type Dummy=1, Losses=553.1320959),
 Row(Policy Number=139895, Number of Vehicles=1, Average Age=42.4, Gender Dummy=0, Married Dummy=0, Avg Veh Age=12.97, Fuel Type Dummy=0, Losses=292.4305792),
 Row(Policy Number=174407, Number of Vehicles=1, Average Age=20.6, Gender Dummy=1, Married Dummy=1, Avg Veh Age=12.97, Fuel Type Dummy=0, Losses=246.5405761),
 Row(Policy Number=170545, Number of Vehicles=1, Average Age=42.4, Gender Dummy=1, Married Dummy=0, Avg Veh Age=8.02, Fuel Type Dummy=0, Losses=384.6763948),
 Row(Policy Number=123164, Number of Vehicles=1, A

In [8]:
insur_data.show()

## To see whether we imported in data frame structure or not

+-------------+------------------+-----------+------------+-------------+-----------+---------------+-----------+
|Policy Number|Number of Vehicles|Average Age|Gender Dummy|Married Dummy|Avg Veh Age|Fuel Type Dummy|     Losses|
+-------------+------------------+-----------+------------+-------------+-----------+---------------+-----------+
|       150023|                 1|       20.6|           1|            0|      12.97|              0|203.3195946|
|       152720|                 1|       42.4|           0|            1|       8.02|              0|402.1973024|
|       174207|                 1|       42.4|           1|            0|        2.5|              1|553.1320959|
|       139895|                 1|       42.4|           0|            0|      12.97|              0|292.4305792|
|       174407|                 1|       20.6|           1|            1|      12.97|              0|246.5405761|
|       170545|                 1|       42.4|           1|            0|       8.02|   

In [58]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors,VectorUDT

In [19]:
### We need to create a tuple of (Label, Features) 
### Where Label is the Y variable and Features is the Dense Vector of all X Variables
### For RDD --> We need to use mllib in import --> Logistic Regression
### For DF --> We need to use ml in import --> Linear Regression 

In [20]:
insurancedf=insur_data.rdd.map(lambda x: [Vectors.dense(x[1:6]),x[-1]]).toDF(['Features','Label'])

In [21]:
insurancedf.show(5)

+--------------------+-----------+
|            Features|      Label|
+--------------------+-----------+
|[1.0,20.6,1.0,0.0...|203.3195946|
|[1.0,42.4,0.0,1.0...|402.1973024|
|[1.0,42.4,1.0,0.0...|553.1320959|
|[1.0,42.4,0.0,0.0...|292.4305792|
|[1.0,20.6,1.0,1.0...|246.5405761|
+--------------------+-----------+
only showing top 5 rows



In [22]:
from pyspark.ml.regression import LinearRegression

In [23]:
lr = LinearRegression(featuresCol='Features',labelCol='Label')

In [24]:
lr_model=lr.fit(insurancedf)

In [41]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [51]:
lr_model.summary.r2


0.5260000072141688

In [52]:
lr_model.summary.rootMeanSquaredError


174.68105816233438

In [53]:
lr_model.coefficients


DenseVector([-2.1789, -9.058, 93.4151, 135.456, -28.6335])

In [54]:
lr_model.intercept

914.1106930643006

In [55]:
lr_model.summary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-244.05524965109933|
| -31.48977828775412|
| 3.4288314874756907|
| 135.93555886204342|
| -336.2903146793871|
|-6.9697390479316255|
|-129.58535505234676|
| 218.05595337214336|
| -409.6493324207121|
|-246.64321499092185|
|  180.8989659206129|
| 115.43830733375569|
|-192.38022346263412|
|  56.85957936216832|
|-33.124428847931654|
|  64.75197717357815|
| 126.99027215539036|
| 189.81392952061287|
| 113.88503796928791|
| -68.87191770242441|
+-------------------+
only showing top 20 rows



In [56]:
lr_model.summary.totalIterations

1

In [61]:
predict=lr_model.transform(insurancedf)

In [62]:
predict.show(5)

+--------------------+-----------+------------------+
|            Features|      Label|        prediction|
+--------------------+-----------+------------------+
|[1.0,20.6,1.0,0.0...|203.3195946| 447.3748442510993|
|[1.0,42.4,0.0,1.0...|402.1973024|433.68708068775413|
|[1.0,42.4,1.0,0.0...|553.1320959| 549.7032644125243|
|[1.0,42.4,0.0,0.0...|292.4305792| 156.4950203379566|
|[1.0,20.6,1.0,1.0...|246.5405761| 582.8308907793871|
+--------------------+-----------+------------------+
only showing top 5 rows



In [63]:
evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='Label',metricName="rmse")

In [64]:
rmse=evaluator.evaluate(predict)

In [65]:
print(rmse)

174.68105816233438


In [66]:
evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='Label',metricName="r2")

In [67]:
r2=evaluator.evaluate(predict)

In [68]:
print(r2)

0.5260000072141688


In [70]:
lr_model.summary.meanAbsoluteError

115.31004699176826

In [71]:
lr_model.summary.meanSquaredError

30513.47208071285

In [72]:
lr_model.summary.pValues

[0.1414190209338151, 0.0, 0.0, 0.0, 0.0, 0.0]

In [73]:
insurancedf1=insur_data.rdd.map(lambda x: [Vectors.dense(x[2:6]),x[-1]]).toDF(['Features1','Label1'])

In [74]:
lr1 = LinearRegression(featuresCol='Features1',labelCol='Label1')

In [75]:
lr_model1=lr1.fit(insurancedf1)

In [76]:
lr_model1.summary.pValues

[0.0, 0.0, 0.0, 0.0, 0.0]

In [77]:
lr_model1.summary.meanSquaredError

30517.78971309719

In [78]:
lr_model1.summary.rootMeanSquaredError

174.69341634159312

In [79]:
lr_model1.summary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -240.8316113028902|
| -28.20925870142969|
|  6.651330664779948|
|  139.1884985809396|
|-333.02717943964836|
| -3.733950583144747|
|-126.31812475350483|
|  221.3339538292693|
| -406.3622756136484|
| -243.3748455211751|
| 184.16210116035165|
| 118.73074394418131|
|-189.15135088441693|
|  60.12042856518505|
|-29.888640383144775|
|  67.99968266246623|
| 130.30253527403784|
| 193.07706476035162|
| 117.17209477635163|
| -65.62435778689019|
+-------------------+
only showing top 20 rows



In [80]:
lr_model1.summary.tValues

[-104.41966771783336,
 33.05313604444258,
 47.90771796100773,
 -79.42246084840689,
 152.68806377954778]

In [81]:
lr_model.summary.tValues

[-1.4706032847815143,
 -104.40700440852557,
 33.05251492798508,
 47.921350426590024,
 -79.4179853381474,
 131.02426378545263]

In [82]:
### Split data into train data and test data

insur_data.count()

15290

In [83]:
traindata,testdata = insurancedf.randomSplit([0.70,0.30],seed=123)

In [85]:
traindata.count()

10659

In [87]:
testdata.count()

4631

In [88]:

lr_model2=lr.fit(traindata)

In [89]:
lr_model2.summary.rootMeanSquaredError

174.06488297135078

In [90]:
lr_model2.summary.r2

0.5346702795957955

In [91]:
lr_model3=lr.fit(testdata)

In [93]:
lr_model3.summary.r2

0.5055331292063

In [94]:
lr_model2.summary.rootMeanSquaredError


174.06488297135078

In [95]:
lr_model2.summary.tValues


[-1.2311650438026973,
 -88.1686458846839,
 28.293956899287092,
 40.54780121517073,
 -67.122185219436,
 110.70903968971584]

In [96]:
lr_model2.summary.pValues

[0.218288321826984, 0.0, 0.0, 0.0, 0.0, 0.0]

In [102]:
### Let us try decision Tree Regression on the same Insurance Data
from pyspark.ml.regression import DecisionTreeRegressor

In [109]:
insurancedf4=insur_data.rdd.map(lambda x: [Vectors.dense(x[2:6]),x[-1]]).toDF(['features','label'])


In [110]:
dtmodel=DecisionTreeRegressor(featuresCol='features')

In [113]:
dt_model=dtmodel.fit(insurancedf4)

In [114]:
dtpred=dt_model.transform(insurancedf4)

In [115]:
dtpred.show(5)

+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[20.6,1.0,0.0,12.97]|203.3195946| 333.2638523903444|
| [42.4,0.0,1.0,8.02]|402.1973024|433.49622672352524|
|  [42.4,1.0,0.0,2.5]|553.1320959| 547.5435941172063|
|[42.4,0.0,0.0,12.97]|292.4305792|249.49646718962936|
|[20.6,1.0,1.0,12.97]|246.5405761| 742.3726205902141|
+--------------------+-----------+------------------+
only showing top 5 rows



In [119]:
evaluator1=RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName="rmse")

In [120]:
evaluator1.evaluate(dtpred)

158.09074509397286

In [124]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer

In [129]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="labelIndex", maxCategories=4).fit(insurancedf4)


In [130]:
pipeline = Pipeline(stages=[featureIndexer,dt_model])

In [131]:
dtmodel2= pipeline.fit(insurancedf4)

In [133]:
predictions = dt_model.transform(insurancedf4)

In [134]:
predictions.select("prediction", "label", "features").show(5)

+------------------+-----------+--------------------+
|        prediction|      label|            features|
+------------------+-----------+--------------------+
| 333.2638523903444|203.3195946|[20.6,1.0,0.0,12.97]|
|433.49622672352524|402.1973024| [42.4,0.0,1.0,8.02]|
| 547.5435941172063|553.1320959|  [42.4,1.0,0.0,2.5]|
|249.49646718962936|292.4305792|[42.4,0.0,0.0,12.97]|
| 742.3726205902141|246.5405761|[20.6,1.0,1.0,12.97]|
+------------------+-----------+--------------------+
only showing top 5 rows



In [135]:
print("Root Mean Squared Error (RMSE) on insurance data = %g" % rmse)

Root Mean Squared Error (RMSE) on insurance data = 174.681


In [137]:
treeModel = dtmodel2.stages[1]

In [138]:
print(treeModel)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4019b8e4cfc2d8f0a2e4) of depth 5 with 55 nodes


In [141]:
treeModel

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4019b8e4cfc2d8f0a2e4) of depth 5 with 55 nodes

In [142]:
### Using Random Forest Regressor

from pyspark.ml.regression import RandomForestRegressor

In [143]:
rfmodel=RandomForestRegressor(featuresCol='features')

In [144]:
rf_model=rfmodel.fit(insurancedf4)

In [145]:
rfpred=rf_model.transform(insurancedf4)

In [146]:
rfpred.show(6)

+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[20.6,1.0,0.0,12.97]|203.3195946| 391.6765875241798|
| [42.4,0.0,1.0,8.02]|402.1973024| 402.9240939715261|
|  [42.4,1.0,0.0,2.5]|553.1320959|  528.436106230095|
|[42.4,0.0,0.0,12.97]|292.4305792|277.73091626397724|
|[20.6,1.0,1.0,12.97]|246.5405761| 647.4706609175017|
| [42.4,1.0,0.0,8.02]|384.6763948|369.05486949260575|
+--------------------+-----------+------------------+
only showing top 6 rows



In [147]:
evaluator1.evaluate(rfpred)

160.1814676325021

In [148]:
display(treeModel)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4019b8e4cfc2d8f0a2e4) of depth 5 with 55 nodes

In [ ]:
print(model.toDebugString())